### BINOMIAL PRICING MODEL FOR SPY CALL OPTION
#### FE-620 FINAL PROJECT

IMPORTS

In [1542]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

BINOMIAL PRICING MODEL FOR AMERICAN OPTION USING RISK NEUTRAL PROBABILITY

In [1543]:
def _get_binom_rn_params(n : float, t : float, r : float, hvol : float, prec : int):
    # calc tree params
    dt = t/n
    u = np.exp(hvol*np.sqrt(dt))
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    q = 1 - p
    # print params
    print((f"dt: {round(dt, 2)}  | u: {round(u, 2)}  |  d: {round(d, 2)}  |  p: {round(p, 2)}  |  q: {round(q, 2)}").center(56)) # 56 is len of divider
    # print divider
    print("-"*55)
    return t, dt, u, d, p, q

In [1544]:
def _print_model(type : str):
    # print divider
    print("-"*55)
    # print model
    print((f"american {type} opt - risk neutral binomial pricing model").center(56)) # 56 is len of divider
    # print divider
    print("-"*55)

In [1545]:
def _get_div_idx(div : dict, steps : int, maturity : float):
    if div is None:
        return None
    # print dividends
    print(f"dividends:{str(div)}".center(56))
    # get "todays" date - time 0
    start = list(div.keys())[0]
    # replace start date with index 0 for start date
    div[0] = div.pop(start)
    # calc time length of each step
    dt = maturity/steps
    # get index of each div
    for date, dividend in div.items():
        # skip start key/val pair
        if date != 0:
            # calculate index aka days since start
            idx = round((((pd.to_datetime(date) - pd.to_datetime(start)).days)/252)/dt)
            # switch date with index
            div[idx] = div.pop(date)
    return div

# explanation of dividend index calculation for varied time steps:
# diff days / 252 = diff in years since start
# dt = (t in years) / steps = time of step in years
# diff in years / time of step in years = steps since start -> rounded = index of step


- need to add dividend adjustment to back propogation - may be worth finding price before loops with dividend adjustment so its separate and easier to just call on later -> more readable
- also need to have adjust _get_div_index to output indices according to steps, not just 122 (daily)

In [1546]:
def _calc_maturity_value(stock : list, tree : list, s0 : float, k : float, n : int, t : float, u : float, d : float, prec : int, type : str, div : dict):
    # call div idx func if divs - will return div to model for subsequent backprop call
    div = _get_div_idx(div, n, t)
    print(f"step dividends:{div}".center(56))
    # check if dividend paid at maturity by checking if # of total steps is in div dict
    if div is not None and n in list(div.keys()):
        # get div amount
        div_adj = div[n]
        # print div adj
        print(f"dividend adjustment: {div_adj}")
    else:
        div_adj = 0
    # check option type
    if type.lower() == "call":
        # iterate over rows (price/nodes) @ maturity
        for i in range(n, -1, -1):
            # print stock price before div adj
            if div_adj != 0 print("price before div adj:", round((s0*(u**i)*(d**(n-i))), prec))
            # calc stock price @ maturity considering potential dividend
            stock[n][i] = round((s0*(u**i)*(d**(n-i)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves - div adj
            # print stock price after div adj
            if div_adj != 0 print("price after div adj:", stock[n][i])
            # calc option price @ maturity considering potential dividend
            tree[n][i] = round(max(stock[n][i]-k, 0), prec) # max(s - d - k, 0)
    elif type.lower() == "put":
        # iterate over rows (price/nodes) @ maturity
        for i in range(n, -1, -1):
            # calc stock price @ maturity considering potential dividend
            stock[n][i] = round((s0*(u**i)*(d**(n-i)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves - div adj
            # calc option price @ maturity considering potential dividend
            tree[n][i] = round(max(k-stock[n][i], 0), prec) # max(k - s, 0)
    else:
        raise ValueError("invalid option type - please enter 'call' or 'put'")
    return stock, tree, div
    

In [1547]:
def _back_prop(stock : list, tree : list, div : dict, s0 : float, k : float, n : int, dt : float, r : float, u : float, d : float, p : float, q : float, prec : int, type : str):
    # iterate over cols (time)
    for i in range(n-1, -1, -1):
        # dividend idxs have already been identified in the maturity calculation so just have to check for them
        # check if div paid at this time step
        if div is not None and i in list(div.keys()):
            # get div amount
            div_adj = div[i]
            # print div adj
            print(f"dividend adjustment: {div_adj}")
        else:
            div_adj = 0
        # iterate over rows (price/nodes)
        for j in range(i+1, -1, -1):
            # print stock price before div adj
            if div_adj != 0 print("price before div adj:", round((s0*(u**i)*(d**(n-i))), prec))
            # fill in stock price at node
            stock[i][j-1] = round((s0*(u**j)*(d**(i-j)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves
            # print stock price after div adj
            if div_adj != 0 print("price after div adj:", stock[i][j-1])
            # find option price at node by discounting expected value of option price at next time step
            tree[i][j-1] = round(np.exp(-r*dt)*(p*tree[i+1][j]+q*tree[i+1][j-1]), prec)
            if type.lower() == "call":
                # # check exercise for call (option price vs intrinsic value) ********************** TODO: finish implementign early exercise print out
                # if max(tree[i][j-1], (stock[i][j-1]) - k) == tree[i][j-1]:
                #     print("call exercised at:", stock[i][j-1])
                tree[i][j-1] = round(max(tree[i][j-1], (stock[i][j-1]) - k), 2) # max(s - k, 0)
            else:
                # check exercise for put (option price vs intrinsic value)
                tree[i][j-1] = round(max(tree[i][j-1], k - (stock[i][j-1])), 2) # max(k - s, 0)
    return stock, tree

In [1548]:
def _print_stock(stock : list):
    # print price
    for time, prices in enumerate(stock, 0):
        centered = str(prices).center(41) # 41 is len of divider after time #:
        print(f"time {time}: {centered}")
    # print divider
    print("-"*55)

In [1549]:
def _print_tree(tree : list):
    # print tree
    for time, branch in enumerate(tree, 0):
        # print centered row
        centered = str(branch).center(41) # 41 is len of divider after time #:
        print(f"time {time}: {centered}")

In [1550]:
def binom_rn_pricer(s0 : float, k : float, n : int, t : float, r : float, hvol : float, prec : int, type : str, div : dict):
    """
    s0: initial stock price
    k: strike price
    n: number of steps
    t: time to maturity per annum
    r: risk-free rate
    hvol: historical volatility - window size = maturity of rfr
    maturity: maturity of rate used (e.g, 1-month = 1, 3-month = 3, 6-month = 6, 1-year = 12)
    type: option type (call or put)
    div: dividend dictionary {start date : pricing date, div1 date : div1 #, div2 date : div2 #, ...}
    """
    # get adjusted & calculated params
    t, dt, u, d, p, q = _get_binom_rn_params(n, t, r, hvol, prec)
    # print model
    _print_model(type)
    # initialize option tree
    tree = [[0]*(i+1) for i in range(n+1)]
    # initialize stock tree
    stock = [[0]*(i+1) for i in range(n+1)]
    # calc option prices at maturity
    stock, tree, div = _calc_maturity_value(stock, tree, s0, k, n, t, u, d, prec, type, div)
    # backpropagate
    stock, tree = _back_prop(stock, tree, div, s0, k, n, dt, r, u, d, p, q, prec, type)
    # print stock price
    _print_stock(stock)
    # print option tree
    _print_tree(tree)
    # return price and df
    return tree[0][0], tree, stock

ALGO TEST

In [1551]:
s0 = 100 # market price as of 6/1/23
k = 101 # call strike
n = 3 # if daily -> biz days to expiration (6/1/23 - 11/17/23) = 122
t = 6/12 # tmt per annum
r = .05 # 5% per annum
hvol = .25 # 10% per annum
prec = 2 # precision
type = "call" # option type
div = None # dividend yield

In [1552]:
price, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=t, r=r, hvol=hvol, prec=prec, type=type, div=div)

dt: 0.16666666666666666  | u: 1.1074522120503245  |  d: 0.9029735000019652  |  p: 0.51543092750191  |  q: 0.48456907249809
-------------------------------------------------------
-------------------------------------------------------
american call opt - risk neutral binomial pricing model 
-------------------------------------------------------
div None
time 0:                  [100.0]                 
time 1:               [110.75, 90.3]             
time 2:           [100.0, 122.65, 81.54]         
time 3:       [73.62, 90.3, 110.75, 135.82]      
-------------------------------------------------------
time 0:                   [9.52]                 
time 1:               [11.07, 8.04]              
time 2:            [0.0, 21.65, 16.73]           
time 3:            [0, 0, 9.75, 34.82]           


READ & PULL DATA

In [1553]:
spyopt = pd.read_excel('spy_opt_data.xlsx')


HISTORICAL VOLATILITY FUNCTION

In [1554]:
def _get_hvol(hist : str, start : str, end : str, prec : int):
    # check if data is already local
    path = './spy_opt_hvol.csv'
    if os.path.isfile(path):
        spy = pd.read_csv(path)
        # download new data if no local data
    else:
        spy = yf.download('SPY', start=hist, end=end)
        spy = spy['Adj Close']
        spy.index = pd.to_datetime(spy.index)
        hist, start, end = pd.to_datetime(hist), pd.to_datetime(start), pd.to_datetime(end)
    # calc historical volatility for window & annualize
    return ((spy.pct_change().loc[hist:start]).dropna().std()*np.sqrt(252)).round(prec)

INITIALIZE MODEL PARAMETERS

In [1555]:


prec = 2 # precision
s0 = 421.82 # market price as of 6/1/23
k = 455 # strike
n = 6 # number of steps
r = .0517# feed in appropriate rate ^
t = n/252 # tmt per annum
type = "call" # option type
hist = '2022-06-01' # 1 year of historical data
start = '2023-06-01' # observation date (i.e, t = 0)
end = '2023-11-17' # expiration date (i.e, t = n)
hvol = _get_hvol(hist, start, end, prec) # get historical volatility
div = {start : hist, start : 1.64, end : 1.58} # use ex-dividend dates

[*********************100%%**********************]  1 of 1 completed


In [1556]:
price, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=t, r=r, hvol=hvol, prec=prec, type=type, div=div)

dt: 0.003968253968253968  | u: 1.013316643673481  |  d: 0.9868583588786172  |  p: 0.5044476995207176  |  q: 0.49555230047928245
-------------------------------------------------------
-------------------------------------------------------
american call opt - risk neutral binomial pricing model 
-------------------------------------------------------
div {0: 0, 15: 1.64, 106: 1.58}
time 0:                  [421.82]                
time 1:              [427.44, 416.28]            
time 2:          [421.82, 433.13, 410.81]        
time 3:      [416.28, 427.44, 438.9, 405.41]     
time 4:  [410.81, 421.82, 433.13, 444.74, 400.08]
time 5: [405.41, 416.28, 427.44, 438.9, 450.66, 394.82]
time 6: [389.63, 400.08, 410.81, 421.82, 433.13, 444.74, 456.67]
-------------------------------------------------------
time 0:                   [0.02]                 
time 1:                [0.0, 0.05]               
time 2:              [0.0, 0.0, 0.1]             
time 3:            [0.0, 0.0, 0.0, 0.2

- make get_hvol() function and have it setup so that it uses previous x window based on frequency 
- incorporate dividends into calcualtion (use dictionary with time step and dividend amount)
- add exercise print out (need to add max() conditional and then print time & executed if intrinsic is greater)

i dont think the dividend is catc